In [12]:
import numpy as np
import json

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer,SnowballStemmer

from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.utils import simple_preprocess

import gensim
import pandas as pd

In [13]:
with open("/home/tyagi/Downloads/Project/annotations_trainval2017/annotations/captions_val2017.json") as file:
    data_val=json.load(file)

In [14]:
# with open("/home/tyagi/Downloads/project_data_bhavin/annotations_trainval2014/annotations/captions_val2014.json") as file:
#     data_val=json.load(file)

In [15]:
print(len(data_val))
#print(data_val["annotations"])
for i in data_val.keys():
    print(i,len(data_val[i]))

4
info 6
licenses 8
images 5000
annotations 25014


In [16]:
#print(data_val["images"])

In [17]:
print(data_val["info"])

{'description': 'COCO 2017 Dataset', 'url': 'http://cocodataset.org', 'version': '1.0', 'year': 2017, 'contributor': 'COCO Consortium', 'date_created': '2017/09/01'}


In [18]:
print(len(data_val['annotations']))
print(data_val['annotations'][1804])

25014
{'image_id': 245764, 'id': 370033, 'caption': 'A cat sitting on the edge of the toilet looking toward the open bathroom door.'}


In [28]:
count=0
for images in data_val['annotations']:
    count +=1
    if images['image_id'] == 245764:
        print(count-1)
        print(images['caption'])

1797
A cat sits on the edge of a toilet.
1804
A cat sitting on the edge of the toilet looking toward the open bathroom door.
1813
a black and gray cat is sitting on a toilet
1814
A cat laying on a toilet seat with its paws inside .
1830
A cat is laying on a toilet seat


In [20]:
stemmer = SnowballStemmer('english')

In [21]:
def lemmatize_stemming(text):
    #return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    #print(text)
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(lemmatize_stemming(token)) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [29]:
count=0
for images in data_val['annotations']:
    if images['image_id'] == 245764:
        result=preprocess(images['caption'])
        #count+=1
        print(result)
        
#print(count)

['edge', 'toilet']
['edge', 'toilet', 'look', 'open', 'bathroom', 'door']
['black', 'gray', 'toilet']
['toilet', 'seat', 'inside']
['toilet', 'seat']


In [23]:
doc_set = []

In [24]:
data_dict={}
for image in data_val['annotations']:
    if image["image_id"] in data_dict:
        data_dict[image["image_id"]].append(image["caption"])
    else:
        data_dict.update({image["image_id"]:[image["caption"]]})


In [25]:
print(len(data_dict)) ### image_id(key) -- > text_corpus(value)

5000


In [30]:
print(data_dict[245764])

['A cat sits on the edge of a toilet.', 'A cat sitting on the edge of the toilet looking toward the open bathroom door.', 'a black and gray cat is sitting on a toilet', 'A cat laying on a toilet seat with its paws inside .', 'A cat is laying on a toilet seat']


In [ ]:
#print(data_dict)

In [31]:
df=pd.DataFrame(data_dict.items(),columns=['image_id','text_corpus'])

In [32]:
df.iloc[3]['text_corpus']

['Two women waiting at a bench next to a street.',
 'A woman sitting on a bench and a woman standing waiting for the bus.',
 'A woman sitting on a bench in the middle of the city',
 'A woman sitting on a bench and a woman standing behind the bench at a bus stop',
 'A woman and another woman waiting at a stop.']

In [33]:
df.head()

,image_id,text_corpus
0,179765,[A black Honda motorcycle parked in front of a...
1,190236,[An office cubicle with four different types o...
2,331352,"[A small closed toilet in a cramped space., A ..."
3,517069,[Two women waiting at a bench next to a street...
4,182417,[A beautiful dessert waiting to be shared by t...


In [34]:
df.shape

(5000, 2)

In [35]:
def lemmatize_stemm(text):
    #return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocessing(Bigtext):
    
    result = []
    for text in Bigtext:
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(lemmatize_stemm(token)) > 3:
                #print(token)
                result.append(lemmatize_stemm(token))
    return result

In [36]:
dict_gensim={}
for i in range(df.shape[0]):
    dict_gensim.update({i:preprocessing(df.iloc[i]["text_corpus"])})

In [37]:
print(dict_gensim[0])
dd=pd.Series(dict_gensim)


['black', 'honda', 'motorcycle', 'park', 'garage', 'honda', 'motorcycle', 'park', 'grass', 'driveway', 'black', 'honda', 'motorcycle', 'dark', 'burgundy', 'seat', 'motorcycle', 'park', 'gravel', 'garage', 'motorcycle', 'brake', 'extend', 'stand', 'outside']


In [38]:
print(type(dd))
print(dd.shape)
dd.head()

<class 'pandas.core.series.Series'>
(5000,)


0    [black, honda, motorcycle, park, garage, honda...
1    [office, cubicle, different, type, computers, ...
2    [small, close, toilet, cramp, space, toilet, s...
3    [women, wait, bench, street, woman, bench, wom...
4    [beautiful, dessert, wait, share, people, piec...
dtype: object

In [39]:
dictionary = gensim.corpora.Dictionary(dd)     

In [40]:
print(dictionary)

Dictionary(4994 unique tokens: ['black', 'brake', 'burgundy', 'dark', 'driveway']...)


In [41]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 black
1 brake
2 burgundy
3 dark
4 driveway
5 extend
6 garage
7 grass
8 gravel
9 honda
10 motorcycle


In [ ]:
#dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=10000)

In [42]:
len(dictionary)

4994

In [43]:
bow_corpus = [dictionary.doc2bow(doc) for doc in dd]

In [44]:
bow_doc_1 = bow_corpus[1]

for i in range(len(bow_doc_1)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0],dictionary[bow_doc_1[i][0]], bow_doc_1[i][1]))

Word 15 ("chair") appears 2 time.
Word 16 ("clutter") appears 1 time.
Word 17 ("computers") appears 1 time.
Word 18 ("cubicle") appears 2 time.
Word 19 ("desk") appears 2 time.
Word 20 ("different") appears 1 time.
Word 21 ("home") appears 1 time.
Word 22 ("laptop") appears 1 time.
Word 23 ("office") appears 5 time.
Word 24 ("screen") appears 1 time.
Word 25 ("space") appears 1 time.
Word 26 ("type") appears 1 time.


In [46]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=30, id2word=dictionary, passes=100)

In [ ]:
#lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=100)

In [47]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.142*"play" + 0.087*"game" + 0.048*"children" + 0.044*"video" + 0.042*"young" + 0.028*"remote" + 0.028*"hold" + 0.027*"sleep" + 0.027*"blanket" + 0.019*"girls"
Topic: 1 
Words: 0.097*"bottle" + 0.071*"plant" + 0.069*"water" + 0.059*"drink" + 0.045*"reach" + 0.042*"bridge" + 0.031*"feed" + 0.030*"milk" + 0.022*"baby" + 0.020*"leave"
Topic: 2 
Words: 0.118*"bird" + 0.059*"airplane" + 0.054*"plane" + 0.042*"large" + 0.036*"rock" + 0.030*"blue" + 0.027*"runway" + 0.024*"white" + 0.024*"small" + 0.022*"airport"
Topic: 3 
Words: 0.100*"stand" + 0.084*"field" + 0.072*"tree" + 0.067*"grass" + 0.036*"giraffe" + 0.035*"fence" + 0.028*"giraffes" + 0.028*"zebra" + 0.028*"zebras" + 0.027*"green"
Topic: 4 
Words: 0.168*"laptop" + 0.133*"desk" + 0.095*"elephant" + 0.055*"keyboard" + 0.049*"monitor" + 0.033*"mouse" + 0.025*"office" + 0.024*"computers" + 0.023*"screen" + 0.020*"table"
Topic: 5 
Words: 0.210*"cake" + 0.032*"candle" + 0.028*"birthday" + 0.024*"pink" + 0.024*"chocolate" 

In [48]:
#lda_model.get_topic_terms(0,topn=30)

In [49]:
#print(lda_model.print_topics(num_topics=10,num_words=30)[1])

In [50]:
lda_model[bow_corpus[4999]]

[(5, 0.28261536), (8, 0.096668996), (15, 0.587378)]

In [51]:
noofTopics=30

In [52]:
final_dict={}
for j in range(len(bow_corpus)):
    li=[0.0]*noofTopics  ### len == no of topics
    probDist=lda_model[bow_corpus[j]]
    for i in range(len(probDist)):
        li[probDist[i][0]]=probDist[i][1]
    final_dict[df.iloc[j]["image_id"]]=li
    

In [53]:
len(final_dict)

5000

In [54]:
#final_dict[397133]

In [55]:
final_dict[139]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.055744477,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5736014,
 0.0,
 0.0,
 0.32779655]

In [56]:
final_dict[1296]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.31480938,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.22091118,
 0.0,
 0.4332334]

In [57]:
import pickle
f=open("ldaResult","wb") 
pickle.dump(final_dict,f)
f.close

<function BufferedWriter.close>

In [58]:
import pickle
with open("Dict","wb") as f:
    pickle.dump(dictionary,f)

In [61]:
topic_word_prob={}
for i in range(noofTopics):
    word_prob={}
    ss=lda_model.get_topic_terms(i,topn=100)
    for x in ss:
        word_prob[dictionary[x[0]]]=x[1]
    topic_word_prob[i]=word_prob

In [62]:
import pickle
with open("topic_word_prob","wb") as f:
    pickle.dump(topic_word_prob,f)

#### tf-idf

In [63]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [64]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=100)

In [65]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))
    print()

Topic: 0 Word: 0.037*"baseball" + 0.035*"horse" + 0.021*"bench" + 0.012*"game" + 0.011*"player" + 0.010*"girl" + 0.010*"woman" + 0.010*"phone" + 0.010*"hold" + 0.010*"ball"

Topic: 1 Word: 0.035*"sign" + 0.030*"street" + 0.022*"wave" + 0.021*"surfboard" + 0.018*"bird" + 0.016*"stop" + 0.014*"light" + 0.014*"surf" + 0.012*"traffic" + 0.011*"city"

Topic: 2 Word: 0.041*"tennis" + 0.037*"train" + 0.024*"clock" + 0.023*"motorcycle" + 0.015*"track" + 0.014*"court" + 0.014*"build" + 0.012*"tower" + 0.012*"park" + 0.012*"street"

Topic: 3 Word: 0.042*"hydrant" + 0.031*"phone" + 0.020*"soccer" + 0.019*"cell" + 0.011*"talk" + 0.006*"bean" + 0.006*"kick" + 0.005*"cellphone" + 0.004*"smart" + 0.003*"sidewalk"

Topic: 4 Word: 0.018*"bear" + 0.018*"field" + 0.018*"stand" + 0.017*"snow" + 0.014*"grass" + 0.013*"frisbee" + 0.012*"giraffe" + 0.011*"tree" + 0.011*"elephant" + 0.010*"sheep"

Topic: 5 Word: 0.044*"skateboard" + 0.019*"trick" + 0.019*"airplane" + 0.018*"plane" + 0.017*"skate" + 0.012*"jum

In [66]:
tf_idf_word_prob={}
for i in range(noofTopics):
    word_prob={}
    ss=lda_model.get_topic_terms(i,topn=100)
    for x in ss:
        word_prob[dictionary[x[0]]]=x[1]
    tf_idf_word_prob[i]=word_prob

In [67]:
import pickle
with open("tf_idf_word_prob","wb") as f:
    pickle.dump(tf_idf_word_prob,f)

In [68]:
for x,y in zip(topic_word_prob[0].keys(),tf_idf_word_prob[0].keys()):
    print(x,topic_word_prob[0][x],y,tf_idf_word_prob[0][y])
    print()

play 0.14164089 play 0.14164089

game 0.08689297 game 0.08689297

children 0.048346743 children 0.048346743

video 0.04423017 video 0.04423017

young 0.041784693 young 0.041784693

remote 0.028066792 remote 0.028066792

hold 0.027923357 hold 0.027923357

sleep 0.027151614 sleep 0.027151614

blanket 0.026827376 blanket 0.026827376

girls 0.019166801 girls 0.019166801

controller 0.017698048 controller 0.017698048

control 0.016172653 control 0.016172653

nintendo 0.014648417 nintendo 0.014648417

room 0.01426607 room 0.01426607

stand 0.014075527 stand 0.014075527

boys 0.013492899 boys 0.013492899

little 0.011925609 little 0.011925609

controllers 0.011293759 controllers 0.011293759

couple 0.011069818 couple 0.011069818

small 0.00811082 small 0.00811082

remotes 0.0076341354 remotes 0.0076341354

screen 0.007387438 screen 0.007387438

child 0.0068025882 child 0.0068025882

women 0.006658541 women 0.006658541

live 0.0066286745 live 0.0066286745

couch 0.006433349 couch 0.006433349

